<img src='./img/DataTailor_EUMETSAT.png'/>

Copyright (c) 2024 EUMETSAT <br>
License: MIT

<hr>

<a href="./index.ipynb">← Index</a>
<br>
<a href="./2_1_Customising_products.ipynb">← Using the EUMETSAT Data Tailor with EUMDAC</a><span style="float:right;"><a href="./3_Data_Tailor_standalone.ipynb">Using the EUMETSAT Data Tailor Standalone with EUMDAC →</a>

# Cleaning the Data Tailor workspace

## What will this module teach you?

This module will show you how to:<br>
1. Retrieve all your customisation jobs
2. Clear a specific job in your workspace
3. Clear all the FAILED jobs in your workspace
4. Clear all the jobs in your workspace

## Prerequisites

In this section we will demonstrate how you can clean up the Data Tailor workspace after having used the Data Tailor web service to customise products selected from the Data Store. More information about this customisation can be found <a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumetsat_data_tailor/-/blob/master/1_Using_the_Data_Tailor_REST_API.ipynb"> here.</a>

We begin, as before by importing our required modules. Notice that we are using the EUMDAC library, a Python interface from EUMETSAT to handle requests and responses of the APIs. We will explain the use of this library in this tutorial.

**We need to install the library first!** An installation guide and further information about the usage of EUMDAC we will find here: https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide

In [ ]:
import eumdac
import datetime
import requests

Now, we have to authorize with our personal credentials to generate the token.

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
You can find your personal API credentials here: <a href="https://api.eumetsat.int/api-key/">https://api.eumetsat.int/api-key/</a>
</div>

In [ ]:
# Insert your personal key and secret into the single quotes

consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET'

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
You can find your personal API credentials here: <a href="https://api.eumetsat.int/api-key/">https://api.eumetsat.int/api-key/</a>
</div>

In [ ]:
try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as error:
    print(f"Error when trying the request to the server: '{error}'")

In [ ]:
datatailor = eumdac.DataTailor(token)

## Cleaning the Data Tailor workspace

### Defining a time window

As mentioned earlier, we need to clean one or several customisation jobs performed with the Data Tailor. However, to be more efficient, we may also want to refine our search in time, so that we could be able to clear at once all the old products obtained over a specific time period. In this example, we will focus on all customisation jobs that are older than one month:

In [ ]:
# Delete all customisations older than one month

delete_datetime = datetime.datetime.combine(datetime.date.today() - datetime.timedelta(days=30), datetime.time(0, 0))

for customisation in datatailor.customisations:
    if customisation.creation_time <= delete_datetime:
        try:
            print(f'Delete customisation {customisation} from {customisation.creation_time} UTC.')
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print(f"Customisation Error:'{error.msg}'")
        except requests.exceptions.RequestException as error:
            print(f"Unexpected error: {error}")
    else:
        print(f"Customisation {customisation} not older than 1 month.")

### Clearing non-running customisations from the Data Tailor

The Data Tailor Web Service has a 20 Gb limit for users, so it is wise to want to clear old products. However, we can choose to clear:
- *all the jobs* at once
- or only a *subset* of these jobs

#### Clearing a single customisation from the Data Tailor

Clearing only one specific customisation product can be done as follows:

In [ ]:
# Individual clearing customisations from the Data Tailor

jobID = '<YOUR_JOBID>'

for customisation in datatailor.customisations:
    if customisation._id == jobID:
        print(f'Customisation {customisation._id} deleted, created on {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except requests.exceptions.RequestException as error:
            print("Unexpected error:", error)

You would then only have to add your **job ID** into the above piece of code to delete the corresponding job from your workspace.

#### Clearing all FAILED customisations from the Data Tailor

But you may also want to clear only all your FAILED customisation products. This can be done as follows: 

In [ ]:
# Clearing all FAILED customisations from the Data Tailor

for customisation in datatailor.customisations:
    if customisation.status == 'FAILED':
        print(f'Delete failed customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except requests.exceptions.RequestException as error:
            print("Unexpected error:", error)

#### Clearing all DONE customisations from the Data Tailor

Should you now want to clean all your DONE customisation products, you could easily adapt the previous lines of code with the corresponding list of jobs:

In [ ]:
# Clearing all DONE customisations from the Data Tailor

for customisation in datatailor.customisations:
    if customisation.status == 'DONE':
        print(f'Delete customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except requests.exceptions.RequestException as error:
            print("Unexpected error:", error)

#### Clearing all (completed) customisations from the Data Tailor

Finally, you may also want to clear all your non running customisation products over this period. This can be done as follows:

In [ ]:
# Clearing all completed customisations from the Data Tailor

for customisation in datatailor.customisations:
    if customisation.status in ['DONE', 'FAILED', 'KILLED', 'DELETED']:
        print(f'Delete completed customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except requests.exceptions.RequestException as error:
            print("Unexpected error:", error)

### Clearing stuck customisations from the Data Tailor
As mentioned before, only the customisation jobs that completed can be removed. However, there exist a way to get rid of running jobs that wouldn't complete (INACTIVE status) or have been queued (QUEUED status).

You may indeed need to clean up your QUEUED jobs or jobs that have been INACTIVE for too long, since they may prevent you from using the Data Tailor services due to the current cap limiting the number of parallel running job to 3.
These QUEUED or INACTIVE jobs however cannot be removed as such. They need to be cancelled first. This can be done as follows :

In [ ]:
# Clearing QUEUED, INACTIVE or RUNNING customisations from the Data Tailor

for customisation in datatailor.customisations:
    if customisation.status in ['QUEUED', 'INACTIVE']:
        customisation.kill()
        print(f'Delete {customisation.status} customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except requests.exceptions.RequestException as error:
            print("Unexpected error:", error)

## Clearing all customisations

Below code will remove all the jobs either they are running, stuck or finished.

In [ ]:
# Clearing all customisations from the Data Tailor

for customisation in datatailor.customisations:
    if customisation.status in ['QUEUED', 'INACTIVE', 'RUNNING']:
        customisation.kill()
        print(f'Delete {customisation.status} customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except Exception as error:
            print("Unexpected error:", error)
    else:
        print(f'Delete completed customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except requests.exceptions.RequestException as error:
            print("Unexpected error:", error)

That ends our example for clearing customisation jobs from the Data Tailor. Feel free to adapt this script to your very specific needs.
If you need further help, you can contact us using the buttons at the bottom of the page.

<a href="./index.ipynb">← Index</a>
<br>
<a href="./2_1_Customising_products.ipynb">← Using the EUMETSAT Data Tailor with EUMDAC</a><span style="float:right;"><a href="./3_Data_Tailor_standalone.ipynb">Using the EUMETSAT Data Tailor Standalone with EUMDAC →</a>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store">View on GitLab</a> | <a href="https://classroom.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>